<a href="https://colab.research.google.com/github/mparker2103/PatternFlow/blob/topic-recognition/recognition/s4436238_UNet/Improved_UNet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, LeakyReLU, BatchNormalization, Dense, Dropout, UpSampling2D, Reshape, concatenate, add
from tensorflow.keras.models import Model
from keras import backend as k

In [6]:
depth = 16
n = 256
batch_size = 20
epochs = 20

In [72]:
def UNet():
    # create the network
    Inputs = Input(shape=(n,n,3), name='Inputs')

    # Downsample 1
    DownConv1 = Conv2D(depth, (3,3), padding='same', name='DownConv1')(Inputs)
    Batch1 = BatchNormalization(name='Batch1')(DownConv1)
    Relu1 = LeakyReLU(alpha=0.01, name='Relu1')(Batch1)
    Dropout1 = Dropout(0.3, name='Dropout1')(Relu1)
    Add1 = add([DownConv1, Dropout1], name='Add1')
    Pool1 = MaxPooling2D(pool_size=(2,2), padding='same', name='Pool1')(Add1)

    # Downsample 2
    DownConv2 = Conv2D(depth*2, (3,3), strides=2, padding='same', name='DownConv2')(Pool1)
    Batch2 = BatchNormalization(name='Batch2')(DownConv2)
    Relu2 = LeakyReLU(alpha=0.01, name='Relu2')(Batch2)
    Dropout2 = Dropout(0.3, name='Dropout2')(Relu2)
    Add2 = add([DownConv2, Dropout2], name='Add2')
    Pool2 = MaxPooling2D(pool_size=(2,2), padding='same', name='Pool2')(Add2)

    # Downsample 3
    DownConv3 = Conv2D(depth*4, (3,3), strides=2, padding='same', name='DownConv3')(Pool2)
    Batch3 = BatchNormalization(name='Batch3')(DownConv3)
    Relu3 = LeakyReLU(alpha=0.01, name='Relu3')(Batch3)
    Dropout3 = Dropout(0.3, name='Dropout3')(Relu3)
    Add3 = add([DownConv3, Dropout3], name='Add3')
    Pool3 = MaxPooling2D(pool_size=(2,2), padding='same', name='Pool3')(Add3)

    # Downsample 4
    DownConv4 = Conv2D(depth*8, (3,3), strides=2, padding='same', name='DownConv4')(Pool3)
    Batch4 = BatchNormalization(name='Batch4')(DownConv4)
    Relu4 = LeakyReLU(alpha=0.01, name='Relu4')(Batch4)
    Dropout4 = Dropout(0.3, name='Dropout4')(Relu4)
    Add4 = add([DownConv4, Dropout4], name='Add4')
    Pool4 = MaxPooling2D(pool_size=(2,2), padding='same', name='Pool4')(Add4)

    # Middle Layer
    MidConv1 = Conv2D(depth*16, (3,3), strides=2, padding='same', name='MidConv1')(Pool4)
    MidConv2 = Conv2D(depth*16, (3,3), strides=2, padding='same', name='MidConv2')(MidConv1)

    # Upsample 4
    UpNet4_0 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet4_0')(MidConv2)
    UpConv4 = Conv2D(depth*8, (3,3), padding='same', name='UpConv4')(UpNet4_0)
    UpNet4_1 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet4_1')(UpConv4)
    Concat4 = concatenate([UpNet4_1, DownConv4], axis=3, name='Concat4')

    # Upsample 3
    UpNet3_0 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet3_0')(Concat4)
    UpConv3 = Conv2D(depth*4, (3,3), padding='same', name='UpConv3')(UpNet3_0)
    UpNet3_1 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet3_1')(UpConv3)
    Concat3 = concatenate([UpNet3_1, DownConv3], axis=3, name='Concat3')

    # Upsample 2
    UpNet2_0 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet2_0')(Concat3)
    UpConv2 = Conv2D(depth*2, (3,3), padding='same', name='UpConv2')(UpNet2_0)
    UpNet2_1 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet2_1')(UpConv2)
    Concat2 = concatenate([UpNet2_1, DownConv2], axis=3, name='Concat2')

    # Upsample 1
    UpNet1_0 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet1_0')(Concat2)
    UpConv1 = Conv2D(depth, (3,3), padding='same', name='UpConv1')(UpNet1_0)
    UpNet1_1 = UpSampling2D(size=(2,2), interpolation='bilinear', name='UpNet1_1')(UpConv1)
    Concat2 = concatenate([UpNet1_1, DownConv1], axis=3, name='Concat1')

    # Reshape into two dimensions
    dense = Dense(1, activation='softmax', name='Dense')(Concat2)
    Outputs = Reshape((n, n), name='Outputs')(dense)

    model = Model(Inputs, Outputs, name='UNet')
    return model

In [73]:
model = UNet()
model.summary()

Model: "UNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Inputs (InputLayer)             [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
DownConv1 (Conv2D)              (None, 256, 256, 16) 448         Inputs[0][0]                     
__________________________________________________________________________________________________
Batch1 (BatchNormalization)     (None, 256, 256, 16) 64          DownConv1[0][0]                  
__________________________________________________________________________________________________
Relu1 (LeakyReLU)               (None, 256, 256, 16) 0           Batch1[0][0]                     
_______________________________________________________________________________________________

In [74]:
# Define dice coefficient metric
def dice(y, y_pred):
    y_flat = k.flatten(y)
    y_pred_flat = k.flatten(y_pred) 
    
    intersect = k.sum(y_flat * y_pred_flat)
    sum = k.sum(k.square(y_flat)) + k.sum(k.square(y_pred_flat))
    return (2 * intersect) / sum